# Влияние компьютерных игр на счастье  
Сейчас мы посчитаем метрики, которые нужны нам для дальнейшнего их анализа.

## Метрики  
Нам нужна метрика, измеряющая время, проводимое испытуемым за компьютером.

Допустим, мы будем измерять время, проводимое пользователем за компьютером, средним количество часов в неделю.  
Назовем ее $GamesHoursPerWeek$ или сокращенно $GHW$

Формула для расчета:  
<p style="text-align: center">$GHW=game\_hours\_per\_day*game\_days\_per\_week$</p>  
Расчитаем метрику для каждой записи и добавим ее в таблицу.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("data_prepared.csv")
df["ghw"] = df.game_hours_per_day * df.game_days_per_week
df.head(5)

,id,age,gender,game_hours_per_day,game_days_per_week,game_type,game_genre,game_friends_activity,game_importance,game_addiction,game_favorite,happiness_general,happiness_personal,happiness_work,happiness_joy,happiness_stress,ghw
0,1855465575,19,M,3,1,Single,Adventure,3,True,False,The Last Of Us,7,3,8,4,2,3
1,1856114803,20,M,4,6,Multiplayer,Action,4,True,True,Deadlock,7,7,4,5,3,24
2,1856117571,19,M,0,1,Multiplayer,Action,3,True,False,Valorant,7,4,7,4,3,0
3,1856208774,20,M,1,2,Multiplayer,Role-Play,4,True,False,Dota 2,7,10,3,4,2,2
4,1856316803,20,M,0,0,Single,Adventure,0,False,False,The Witcher 3,8,9,7,4,1,0


Теперь, нам нужна метрика, отслеживающая уровень счастья.  
Здесь все обстоит несколько сложнее. Метрика для отслеживания игрового времени довольно интуитивна, в то время как метрику для отслеживания счастья трудно подобрать. Какие-то факторы влияют сильнее, какие-то меньше. Поэтому, мы дадим каждому фактору *вес*, а наша метрика будет состоять из сумм произведения веса каждого фактора на его значение.

Однако, наши факторы измеряются в разных шкалах (от 1 до 10 и от 1 до 5), поэтому, предварительно мы приведем все факторы к одной шкале.  
Для этого воспользуемся следующей формулой:  
<p style="text-align: center">$X_{norm} = (X-X_{min})/(X_{max}-X_{min})$</p>  
Также, округлим все значения до 2 знаков после запятой.

In [3]:
df["happiness_general_norm"] = round((df.happiness_general - 1) / (10-1), 2).astype(float)
df["happiness_personal_norm"] = round((df.happiness_personal - 1) / (10-1), 2).astype(float)
df["happiness_work_norm"] = round((df.happiness_work - 1) / (10-1), 2).astype(float)
df["happiness_joy_norm"] = round((df.happiness_joy - 1) / (5-1), 2).astype(float)
df["happiness_stress_norm"] = round((df.happiness_stress - 1) / (5-1), 2).astype(float)
df.iloc[:, 11:]

,happiness_general,happiness_personal,happiness_work,happiness_joy,happiness_stress,ghw,happiness_general_norm,happiness_personal_norm,happiness_work_norm,happiness_joy_norm,happiness_stress_norm
0,7,3,8,4,2,3,0.67,0.22,0.78,0.75,0.25
1,7,7,4,5,3,24,0.67,0.67,0.33,1.00,0.50
2,7,4,7,4,3,0,0.67,0.33,0.67,0.75,0.50
3,7,10,3,4,2,2,0.67,1.00,0.22,0.75,0.25
4,8,9,7,4,1,0,0.78,0.89,0.67,0.75,0.00
5,7,10,8,5,5,21,0.67,1.00,0.78,1.00,1.00
6,7,2,5,4,1,12,0.67,0.11,0.44,0.75,0.00
7,10,7,10,5,3,35,1.00,0.67,1.00,1.00,0.50
8,6,6,6,3,3,12,0.56,0.56,0.56,0.50,0.50
9,8,9,8,3,3,14,0.78,0.89,0.78,0.50,0.50


Теперь, когда мы привели все метрик к одной шкале, расчитаем нашу метрику в соответствии с данным весами.  
| № | Фактор             | Вес  |
| - | -----              | ---  |
| 1 | happiness_general  | 0.4  |
| 2 | happiness_personal | 0.2  |
| 3 | happiness_work     | 0.2  |
| 4 | happiness_joy      | 0.1  |
| 5 | happiness_stress   | -0.1 |

Назовем нашу метрику $HappinessIndex$.  
<p style="text-align: center">$HappinessIndex = \sum_{i=1}^{5}weight_i*factor_i$</p>
Также, округим метрику до 2 знаков после запятой.

In [4]:
df["happiness_index"] = round(df.happiness_general_norm * 0.4 + \
                        df.happiness_personal_norm * 0.2 + \
                        df.happiness_work_norm * 0.2 + \
                        df.happiness_joy_norm * 0.1 + \
                        df.happiness_stress_norm * -0.1, 2).astype(float)
df.iloc[:, 11:]

,happiness_general,happiness_personal,happiness_work,happiness_joy,happiness_stress,ghw,happiness_general_norm,happiness_personal_norm,happiness_work_norm,happiness_joy_norm,happiness_stress_norm,happiness_index
0,7,3,8,4,2,3,0.67,0.22,0.78,0.75,0.25,0.52
1,7,7,4,5,3,24,0.67,0.67,0.33,1.00,0.50,0.52
2,7,4,7,4,3,0,0.67,0.33,0.67,0.75,0.50,0.49
3,7,10,3,4,2,2,0.67,1.00,0.22,0.75,0.25,0.56
4,8,9,7,4,1,0,0.78,0.89,0.67,0.75,0.00,0.70
5,7,10,8,5,5,21,0.67,1.00,0.78,1.00,1.00,0.62
6,7,2,5,4,1,12,0.67,0.11,0.44,0.75,0.00,0.45
7,10,7,10,5,3,35,1.00,0.67,1.00,1.00,0.50,0.78
8,6,6,6,3,3,12,0.56,0.56,0.56,0.50,0.50,0.45
9,8,9,8,3,3,14,0.78,0.89,0.78,0.50,0.50,0.65


Теперь, у нас есть метрики:  
- $GameHoursPerWeek$ - отвечающая за время за играми;  
- $HappinessIndex$ - отвечающая за уровень счастья.

С их помощью мы сможем узнать, есть ли связь между количеством часов за играми и счастьем.

In [5]:
df.to_csv("data_metrics.csv", index=False)